In [1]:
# Core
from pathlib import Path
import gzip
from warnings import warn
import os
import time

# 3rd party
from wget import download
import numpy as np
import pandas as pd
import astropy.units as u
import ephem

In [2]:
def min_magnitude_estimate(minor_planet):
    return minor_planet.H + 2.5 * np.log(minor_planet.Perihelion_dist**2 * (1 - minor_planet.Perihelion_dist)**2)

In [3]:
class MinorPlanetDB():
    def __init__(self,
                 path='../data/mpcorb_extended.json.gz',
                 download_url='http://minorplanetcenter.net/Extended_Files/mpcorb_extended.json.gz',
                 max_age=1 * u.day):
        
        self.path = Path(path)
        self.url = download_url
        self.max_age = max_age
        
        self.refresh_db()
                    
    def refresh_db(self):
        
        # Check if MPCORB database already downloaded. If not attempt to download it
        if not self.path.exists():
            warn("MPC Orbit Database file not found. Downloading...")
            download(self.url, out=self.path.as_posix())
            warn("Done.")
        # Check age of MPCORB database. If exceeds max_age download again
        elif (time.time() - os.path.getmtime(self.path.as_posix())) * u.second > self.max_age:
            warm("MPC Orbit Database file stale. Downloading a fresh copy...")
            download(self.url, out=self.path.as_posix())
            warn("Done.")
            
        # Load MPCORB database as a pandas DataFrame
        with gzip.open(self.path.as_posix(), 'rt') as db:
            self.db = pd.read_json(db)
            
        # Create a unique ID column from Number & Name (if present) or Principal_desig if not
        self.db['ID'] = (self.db.Number + ' ' + self.db.Name).combine_first(self.db.Principal_desig)
        
        # Set unique ID as the index
        self.db.set_index('ID', inplace=True)
        
    def make_bodies(self, magnitude_limit=12):
        
        # Filter minor planets that definitely would be fainter than magnitude limit
        pruned_db = self.db[np.logical_or(lambda x: x.Perihelion_dist < 1.0, 
                                          lambda x: max_magnitude_estimate(x) < magnitude_limit)]
        
        self.bodies = {}
        for ID, minor_planet in pruned_db.iterrows():
            body = ephem.EllipticalBody()
            body.name = ID
            body._inc = minor_planet.i
            body._Om = minor_planet.Node
            body._om = minor_planet.Peri
            body._a = minor_planet.a
            body._M = minor_planet.M
            body._epoch_M = minor_planet.Epoch
            body._e = minor_planet.e
            body._epoch = minor_planet.Epoch
            body._H = minor_planet.H
            body._G = minor_planet.G
            
            self.bodies[ID] = body

In [4]:
minor_planets = MinorPlanetDB()

In [5]:
minor_planets.db.info()

<class 'pandas.core.frame.DataFrame'>
Index: 747239 entries, (1) Ceres to 5154 T-3
Data columns (total 38 columns):
Aphelion_dist                         747239 non-null float64
Arc_length                            139560 non-null float64
Arc_years                             607679 non-null object
Computer                              747239 non-null object
Critical_list_numbered_object_flag    578 non-null float64
Epoch                                 747239 non-null float64
G                                     743886 non-null float64
H                                     743886 non-null float64
Hex_flags                             747239 non-null object
Last_obs                              747239 non-null object
M                                     747239 non-null float64
NEO_flag                              16949 non-null float64
Name                                  21111 non-null object
Node                                  747239 non-null float64
Num_obs                   

In [6]:
minor_planets.make_bodies()

In [7]:
minor_planets.bodies['(1566) Icarus']

<ephem.EllipticalBody '(1566) Icarus' at 0x7f947f752df0>

In [8]:
minor_planets.bodies['(1566) Icarus'].compute()

In [11]:
minor_planets.bodies['(1566) Icarus'].a_ra

3.651808049117752